<a href="https://colab.research.google.com/github/JMSBornot/PytorchCourseDL/blob/main/simple_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [86]:
# Read MNIS images data into Tensors
ttfunc = transforms.ToTensor() # transform to tensor function
train_data = datasets.MNIST(root='input_data', train=True, download=True, transform=ttfunc)
test_data = datasets.MNIST(root='input_data', train=False, download=True, transform=ttfunc)
print(train_data)
print(test_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: input_data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: input_data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [87]:
# Create CNN model
cnv1 = nn.Conv2d(1, 6, 3, 1) # 1 indicates 1 image as input, 6 for channels/filters, 3x3 for kernel size, and stride=1 in this order
cnv2 = nn.Conv2d(6, 16, 3, 1) # input are the 6 channels/filters from cnv1, and it uses 16 channels/filters with 3x3 kernel

In [88]:
# Grab 1 MNIST record/image
for i, (X_train, y_train) in enumerate(train_data):
  break
X_train.shape
x = X_train.view(1,1,28,28) # reshape the tensor
print(x.shape)

torch.Size([1, 1, 28, 28])


In [89]:
# Perform the first convolution operation
x = F.relu(cnv1(x))
print(x.shape)
# Pass through the pooling layer
x = F.max_pool2d(x,2,2)
print(x.shape)
# Second convolution operation
x = F.relu(cnv2(x))
print(x.shape)
# Pass through the pooling layer
x = F.max_pool2d(x,2,2)
print(x.shape)

torch.Size([1, 6, 26, 26])
torch.Size([1, 6, 13, 13])
torch.Size([1, 16, 11, 11])
torch.Size([1, 16, 5, 5])


In [90]:
# Model class
class CANN(nn.Module):
  def __init__(self):
    super().__init__() # always first instantiate the ancestor class
    # covolution layers separated by pooling layers
    self.cnv1 = nn.Conv2d(1,6,3,1)	# input 1x1x28x28 -> 1x6x26x26
    self.cnv2 = nn.Conv2d(6,16,3,1)	# input 1x6x13x13 -> 1x16x11x11 (between cnv1 and cnv2 layers, there is a dropout layer reducing the size 50%)
    # fully connected layers stacked at the end, receiving flattened convolution outcome
    self.fc1 = nn.Linear(16*5*5,120)	# input 1x16x5x5 -> 1x120 (between cnv2 and fc1 layers, there is a dropout layer reducing the size 50%)
    self.fc2 = nn.Linear(120,84)	# input 1x120 -> 1x84
    self.fc3 = nn.Linear(84,10)		# input 1x84 -> 1x10

  def forward(self, x):
    # Convolution layers processing
    x = F.relu(self.cnv1(x))
    x = F.max_pool2d(x,2,2)
    x = F.relu(self.cnv2(x))
    x = F.max_pool2d(x,2,2)

    # flatten conv layers output
    x = x.view(-1,16*5*5) # treating the first dimension as unknown (-1) so we can vary the batch size

    # Fully connected layers processing
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return F.log_softmax(x,dim=1)

In [91]:
# Create and instance and use the model
torch.manual_seed(41)
model = CANN()
model

CANN(
  (cnv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (cnv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [92]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [93]:
import time

# Control variables
Ne = 10                # Number of epochs
Nb = 20               # Batch size
trainLoss = []
testLoss = []
trainAcc = []
testAcc = []

train_loader = DataLoader(train_data, batch_size=Nb, shuffle=True)
test_loader = DataLoader(test_data, batch_size=Nb, shuffle=False)

Ntr = train_loader.dataset.train_data.size(0)
Nte = test_loader.dataset.test_data.size(0)

### Training CNN model
T0 = time.time()

for i in range(Ne): # epochs loop
  acc_train = 0
  acc_test = 0
  num = 0

  # Evaluate for each batch's samples
  for j, (Xtr, ytr) in enumerate(train_loader):
    # Propagate forward and calculate the loss
    out = model(Xtr)  # not flattenet: dim is Nb x 10
    loss = criterion(out, ytr)

    # Accumulate classification accuracy
    _, ind = torch.max(out.data, 1, keepdim=True) # ind's dimension is Nb x 1. If only one output argument is specified then the output will be a tuple (max_value, ind)
    acc_train += (ytr.reshape(-1,1) == ind.reshape(-1,1)).sum()

    # Update CNN model parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Learning feedback
    if (j+1) % 500 == 0: # it runs for 3000 iterations: 3000 x (20 batch samples) = 60,000 MNIST images
      print(f"Epoch {i}, Batch {j+1}: Loss = {loss.data}")

  trainLoss.append(loss)
  trainAcc.append(acc_train)

  # Evaluation on test data
  with torch.no_grad(): # switch off gradient calculation for efficiency
    for j, (Xte, yte) in enumerate(test_loader):
      out = model(Xte)
      _, ind = torch.max(out.data, 1, keepdim=True)
      acc_test += (yte.reshape(-1,1) == ind.reshape(-1,1)).sum()

  loss = criterion(out, yte)
  testLoss.append(loss)
  testAcc.append(acc_test)

  print(f"Epoch {i}, Train_Loss = {trainLoss[-1]}, Test_Loss = {testLoss[-1]}, Train_Acc = {trainAcc[-1]/Ntr}, Test_Acc = {testAcc[-1]/Nte}")

total =  time.time() - T0
print(f"Training took: {total/60} minutes!")

Epoch 0, Batch 500: Loss = 0.6200645565986633
Epoch 0, Batch 1000: Loss = 0.32105380296707153
Epoch 0, Batch 1500: Loss = 0.31112587451934814
Epoch 0, Batch 2000: Loss = 0.018337208777666092
Epoch 0, Batch 2500: Loss = 0.09715411067008972
Epoch 0, Batch 3000: Loss = 0.007037168834358454
Epoch 0, Train_Loss = 0.007037168834358454, Test_Loss = 0.2056254893541336, Train_Acc = 0.9263499975204468, Test_Acc = 0.9760000109672546
Epoch 1, Batch 500: Loss = 0.018876830115914345
Epoch 1, Batch 1000: Loss = 0.4013398289680481
Epoch 1, Batch 1500: Loss = 0.0208301804959774
Epoch 1, Batch 2000: Loss = 0.006143363658338785
Epoch 1, Batch 2500: Loss = 0.02522009238600731
Epoch 1, Batch 3000: Loss = 0.1259872019290924
Epoch 1, Train_Loss = 0.1259872019290924, Test_Loss = 0.07093603909015656, Train_Acc = 0.9777166843414307, Test_Acc = 0.9825000166893005
Epoch 2, Batch 500: Loss = 0.0006617597537115216
Epoch 2, Batch 1000: Loss = 0.023345859721302986
Epoch 2, Batch 1500: Loss = 0.006193316075950861
Epoc